In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


In [2]:
train = pd.DataFrame(pd.read_csv("/kaggle/input/playground-series-s4e7/train.csv"))

In [3]:
test = pd.DataFrame(pd.read_csv("/kaggle/input/playground-series-s4e7/test.csv"))

In [4]:
train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [5]:
train['Region_Code'] = train['Region_Code'].astype('int16')
train['Policy_Sales_Channel'] = train['Policy_Sales_Channel'].astype('int16')

In [6]:
X = train.drop(['Response', 'id'], axis=1)

In [7]:
y = train['Response']

In [8]:
X.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,Male,21,1,35,0,1-2 Year,Yes,65101.0,124,187
1,Male,43,1,28,0,> 2 Years,Yes,58911.0,26,288
2,Female,25,1,14,1,< 1 Year,No,38043.0,152,254
3,Female,35,1,1,0,1-2 Year,Yes,2630.0,156,76
4,Female,36,1,15,1,1-2 Year,No,31951.0,152,294


In [9]:
cat_features = [0, 2, 3, 4, 5, 6, 8]

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=22)

In [11]:
import gc
# 가비지 컬렉션 수행 전, 객체 수 확인
print("Before GC:", len(gc.get_objects()))

# 가비지 컬렉션 수행
gc.collect()

# 가비지 컬렉션 수행 후, 객체 수 확인
print("After GC:", len(gc.get_objects()))

Before GC: 186825
After GC: 186236


In [20]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier(
    iterations=20,
    learning_rate=0.2,
    depth=8,
    one_hot_max_size=3,
    custom_loss=['AUC', 'Recall']
)

clf.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_val, y_val),
)

0:	learn: 0.5768020	test: 0.5770165	best: 0.5770165 (0)	total: 3.16s	remaining: 1m
1:	learn: 0.4990899	test: 0.4994573	best: 0.4994573 (1)	total: 5.96s	remaining: 53.6s
2:	learn: 0.4440197	test: 0.4445033	best: 0.4445033 (2)	total: 8.55s	remaining: 48.4s
3:	learn: 0.4038148	test: 0.4043958	best: 0.4043958 (3)	total: 11.6s	remaining: 46.4s
4:	learn: 0.3734795	test: 0.3741352	best: 0.3741352 (4)	total: 14.2s	remaining: 42.6s
5:	learn: 0.3504816	test: 0.3511918	best: 0.3511918 (5)	total: 16.8s	remaining: 39.1s
6:	learn: 0.3327711	test: 0.3335209	best: 0.3335209 (6)	total: 19.3s	remaining: 35.9s
7:	learn: 0.3190376	test: 0.3198199	best: 0.3198199 (7)	total: 22s	remaining: 33s
8:	learn: 0.3082480	test: 0.3090727	best: 0.3090727 (8)	total: 24.6s	remaining: 30s
9:	learn: 0.2997385	test: 0.3005976	best: 0.3005976 (9)	total: 27.3s	remaining: 27.3s
10:	learn: 0.2929639	test: 0.2938502	best: 0.2938502 (10)	total: 29.9s	remaining: 24.4s
11:	learn: 0.2876085	test: 0.2885145	best: 0.2885145 (11)	tot

In [21]:
y_pred = clf.predict(data=X_val)

In [22]:
acc = y_val == y_pred

In [23]:
i = 0
for d in acc:
    if d == True:
        i+=1
i/acc.size

0.8765028509839371

In [24]:
s, n = 0, 0
for j in y_pred:
    n+=1
    if j==1:
        s += 1
print(s, n)

583 1150480
